In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import shap
from dask_ml.model_selection import train_test_split as dask_train_test_split
from dask.distributed import Client
import dask.dataframe as dd

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import random
from scipy.stats import f_oneway, kruskal
from statsmodels.stats.multitest import multipletests
import seaborn as sns

In [8]:
hp_tr_4 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_tr_4.csv")
hp_tr_24 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_tr_24.csv")
hp_tr_48 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_tr_48.csv")
hp_tr_72 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_tr_72.csv")

hp_mo_4 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_mo_4.csv")
hp_mo_24 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_mo_24.csv")
hp_mo_48 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_mo_48.csv")
hp_mo_72 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_hp_mo_72.csv")

lp_tr_4 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_tr_4.csv")
lp_tr_24 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_tr_24.csv")
lp_tr_48 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_tr_48.csv")
lp_tr_72 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/loopless_flux_samples_lp_tr_72.csv")

lp_mo_4 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_mo_4.csv")
lp_mo_24 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_mo_24.csv")
lp_mo_48 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_mo_48.csv")
lp_mo_72 = pd.read_csv("/home/users/lzehetner/data/paper4_aav/HEK_AAV/flux_sampling/adapted_bm/loopless_flux_samples_lp_mo_72.csv")


In [35]:
def kruskal_wallis_enrichment(df1, df2, rows_df1, rows_df2, pval_threshold=0.05, include_fold_change=True):
    """
    Perform Kruskal-Wallis test between selected rows from two dataframes
    and return significantly enriched reactions after multiple test correction,
    along with upregulated and downregulated features based on fold change.

    Parameters:
    - df1 (pd.DataFrame): First dataframe.
    - df2 (pd.DataFrame): Second dataframe.
    - rows_df1 (list or slice): Row indices to select from df1.
    - rows_df2 (list or slice): Row indices to select from df2.
    - pval_threshold (float): Significance threshold for corrected p-values (default: 0.05).
    - include_fold_change (bool): If True, calculate fold changes for significantly enriched features.

    Returns:
    - dict: Contains three DataFrames:
        1. 'significant_features': All significant IDs with adjusted p-values.
        2. 'upregulated': Significantly upregulated IDs with fold changes.
        3. 'downregulated': Significantly downregulated IDs with fold changes.
    """
    # Select the rows for each dataframe
    data1 = df1.iloc[rows_df1, :]
    data2 = df2.iloc[rows_df2, :]

    # Initialize a list to store p-values
    p_values = []
    fold_changes = []

    # Perform Kruskal-Wallis test for each column
    for col in data1.columns:
        try:
            # Perform Kruskal-Wallis test between selected rows of df1 and df2
            _, p = kruskal(data1[col], data2[col])
            p_values.append(p)

            # Calculate fold change if required
            if include_fold_change:
                mean1 = data1[col].mean()
                mean2 = data2[col].mean()
                fold_change = mean1 / mean2 if mean2 != 0 else np.inf  # Avoid division by zero
                fold_changes.append(fold_change)
            else:
                fold_changes.append(None)
        except:
            # Handle cases where Kruskal-Wallis test fails
            p_values.append(np.nan)
            fold_changes.append(None)

    # Create a DataFrame for results
    results = pd.DataFrame({
        'Feature': data1.columns,
        'p_value': p_values,
        'fold_change': fold_changes if include_fold_change else None
    })

    # Remove rows with NaN p-values
    results = results.dropna(subset=['p_value'])

    # Perform multiple testing correction
    corrected_p_values = multipletests(results['p_value'], method='fdr_bh')[1]
    results['adjusted_p_value'] = corrected_p_values

    # Filter for significant features
    significant_features = results[results['adjusted_p_value'] < pval_threshold]

    # Separate upregulated and downregulated features
    if include_fold_change:
        upregulated = significant_features[significant_features['fold_change'] > 1]
        downregulated = significant_features[significant_features['fold_change'] < 1]
    else:
        upregulated = pd.DataFrame()
        downregulated = pd.DataFrame()

    # Return all results as a dictionary
    return {
        'significant_features': significant_features,
        'upregulated': upregulated,
        'downregulated': downregulated
    }

In [26]:
tr_4 = kruskal_wallis_enrichment(hp_tr_4, lp_tr_4, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)
tr_24 = kruskal_wallis_enrichment(hp_tr_24, lp_tr_24, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)
tr_48 = kruskal_wallis_enrichment(hp_tr_48, lp_tr_48, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)
tr_72 = kruskal_wallis_enrichment(hp_tr_72, lp_tr_72, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)

mo_4 = kruskal_wallis_enrichment(hp_mo_4, lp_mo_4, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)
mo_24 = kruskal_wallis_enrichment(hp_mo_24, lp_mo_24, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)
mo_48 = kruskal_wallis_enrichment(hp_mo_48, lp_mo_48, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)
mo_72 = kruskal_wallis_enrichment(hp_mo_72, lp_mo_72, slice(0, 1000), slice(0, 1000), pval_threshold=0.05, include_fold_change=True)

In [28]:
def find_common_features(results_list, key='upregulated'):
    """
    Find common features present in all results dictionaries for a given key.

    Parameters:
    - results_list (list): A list of results dictionaries.
    - key (str): The key to extract features from ('upregulated' or 'downregulated').

    Returns:
    - list: Features common to all results in the specified key.
    """
    # Initialize a set with features from the first dictionary
    common_features = set(results_list[0][key]['Feature'])

    # Find the intersection with features from other dictionaries
    for results in results_list[1:]:
        common_features &= set(results[key]['Feature'])

    return list(common_features)

# Example usage
# Assuming `results_list` contains 8 dictionaries from the kruskal_wallis_enrichment function
results_list = [tr_4, tr_24, tr_48, tr_72, mo_4, mo_24, mo_48, mo_72]  # Replace with actual variables

# Find features common to all upregulated results
common_upregulated = find_common_features(results_list, key='upregulated')

# Find features common to all downregulated results
common_downregulated = find_common_features(results_list, key='downregulated')

In [29]:
# Upregulated in HP
common_upregulated
# 'MAR00648':    CHKB, CHKA, ETNK1, ETNK2    ATP [c] + ethanolamine [c] ⇒ ADP [c] + H+ [c] + ethanolamine-phosphate [c]
# 'MAR09062':    -                           asparagine [e] ⇔
# 'MAR05015':    -                           (R)-3-hydroxybutanoate [m] + H+ [i] ⇒ (R)-3-hydroxybutanoate [c] + H+ [m]
# 'MAR02267':    ELOVL5, ELOVL1, ELOVL4, ELOVL3, ELOVL7, ELOVL6, ELOVL2   H+ [c] + heneicosanoyl-CoA [c] + malonyl-CoA [c] ⇒ 3-oxotricosanoyl-CoA [c] + CO2 [c] + CoA [c] 
# 'MAR02270':    TECR                        (2E)-tricosenoyl-CoA [c] + H+ [c] + NADPH [c] ⇒ NADP+ [c] + tricosanoyl-CoA [c]
# 'MAR02269':    HACD3, HIGD2A, HACD1, HACD4, HACD2   3-hydroxytricosanoyl-CoA [c] ⇒ (2E)-tricosenoyl-CoA [c] + H2O [c]
# 'MAR02268':    HSD17B12                    3-oxotricosanoyl-CoA [c] + H+ [c] + NADPH [c] ⇒ 3-hydroxytricosanoyl-CoA [c] + NADP+ [c]
# 'MAR09135':    -                           L-lactate

['MAR00648',
 'MAR09062',
 'MAR05015',
 'MAR02267',
 'MAR02270',
 'MAR02269',
 'MAR02268',
 'MAR09135']

In [30]:
## Upregulated in LP
common_downregulated
# 'MAR11443':    MPC1, MPC2    H+ [m] + phenylpyruvate [m] ⇔ H+ [c] + phenylpyruvate [c]
# 'MAR01894':    SLC7A9, SLCO1A2, SLCO1B3    HCO3- [c] + S-glutathionyl-ethacrynic acid [c] + taurochenodeoxycholate [e] ⇔ HCO3- [e] + S-glutathionyl-ethacrynic acid [e] + taurochenodeoxycholate [c]
# 'MAR04562':    CPT1A, CPT1C, CPT1B    L-carnitine [c] + myristoyl-CoA [c] ⇔ CoA [c] + tetradecanoylcarnitine [c]
# 'MAR04722':    CROT    L-carnitine [x] + hexanoyl-CoA [x] ⇔ CoA [x] + Hexanoyl Carnitine [x]
# 'MAR01779':    -     3alpha,7alpha,12alpha-trihydroxy-5beta-cholestanate [m] ⇔ 3alpha,7alpha,12alpha-trihydroxy-5beta-cholestanate [x]
# 'MAR03209':    CYP27A1    5beta-cholestane-3alpha,7alpha,12alpha,26-tetrol [m] + NAD+ [m] ⇔ 3alpha,7alpha,12alpha-trihydroxy-5beta-cholestan-27-al [m] + H+ [m] + NADH [m]
# 'MAR09071':    -     glutamate [e] ⇔
# 'MAR04726':    SLC25A20    Hexanoyl Carnitine [x] ⇔ Hexanoyl Carnitine [c]
# 'MAR04436':    SLCO1A2, SLCO1B3     GSH [c] + HCO3- [c] + histamine [e] ⇔ GSH [e] + HCO3- [e] + histamine [c]
# 'MAR04109':    GPT2    AKG [m] + alanine [m] ⇔ glutamate [m] + pyruvate [m]
# 'MAR01623':    -      3alpha,7alpha,12alpha-trihydroxy-5beta-cholestanate [c] ⇔ 3alpha,7alpha,12alpha-trihydroxy-5beta-cholestanate [x]
# 'MAR02404':    SLC22A10    10-HETE [e] ⇔ 10-HETE [c]
# 'MAR06109':    SLCO1A2, SLCO1B3    HCO3- [c] + S-glutathionyl-2-4-dinitrobenzene [c] + bilirubin [e] ⇔ HCO3- [e] + S-glutathionyl-2-4-dinitrobenzene [e] + bilirubin [c]
# 'MAR04028':    NME1-NME2,NME3,NME4,NME7,NME6,NME1,NME2    ADP [c] + CTP [c] ⇔ ATP [c] + CDP [c]
# 'MAR04152':    SUCLG1,SUCLG2,SUCLA2,SUCLG1,    ADP [m] + Pi [m] + succinyl-CoA [m] ⇔ ATP [m] + CoA [m] + succinate [m]
# 'MAR06041':    SLC16A8,SLC16A6,SLC16A7,SLC16A3,SLC16A1,SLC16A4,SLC16A5    (R)-3-hydroxybutanoate [c] + formate [e] ⇔ (R)-3-hydroxybutanoate [e] + formate [c]
# 'MAR04477':    RPE, RPEL1    D-xylulose-5-phosphate [c] ⇔ ribulose-5-phosphate [c]
# 'MAR05536':    SLC7A5, ABCA1    phenylalanine [c] + proline [e] ⇔ phenylalanine [e] + proline [c]
# 'MAR02608':    CPT1A,CPT1C,CPT1B    CoA [c] + tetradecanoylcarnitine [c] ⇔ L-carnitine [c] + myristoyl-CoA [c]
# 'MAR01616':    GOT2, TAT    AKG [m] + phenylalanine [m] ⇔ glutamate [m] + phenylpyruvate [m]
# 'MAR07802':    AK5     2 dADP [c] ⇔ dAMP [c] + dATP [c]
# 'MAR04312':    ARCN1,COPE,COPB1,COPG2,COPG1,COPB2,    D-glucitol [c] ⇔ D-glucitol [e]
# 'MAR09068':    -     proline [e] ⇔
# 'MAR04719':    CPT1A,CPT1C,CPT1B    L-carnitine [c] + hexanoyl-CoA [c] ⇔ CoA [c] + Hexanoyl Carnitine [c]

['MAR11443',
 'MAR01894',
 'MAR04562',
 'MAR04722',
 'MAR01779',
 'MAR03209',
 'MAR09071',
 'MAR04726',
 'MAR04436',
 'MAR04109',
 'MAR01623',
 'MAR02404',
 'MAR06109',
 'MAR04028',
 'MAR04152',
 'MAR06041',
 'MAR04477',
 'MAR05536',
 'MAR02608',
 'MAR01616',
 'MAR07802',
 'MAR04312',
 'MAR09068',
 'MAR04719']

In [158]:
calculate_mean_std(hp_tr_4, "MAR09062")

Mean of MAR09062: -0.005376158486441645
Standard Deviation of MAR09062: 7.960151658549377e-05
